# LSTM Model

In [1]:
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Dense, LSTM, TimeDistributed, Input

Using TensorFlow backend.


In [2]:
lstm_train = np.load('processed_input/train_features_lstm.npz')
lstm_test = np.load('processed_input/test_features_lstm.npz')

X = pd.read_csv('processed_input/train_lstm.csv.gz')
X_test = pd.read_csv('processed_input/test_lstm.csv.gz')
df_to_reshape = pd.read_csv('processed_input/prep_submission_lstm.csv.gz')
submission_lstm = pd.read_csv('processed_input/submission_lstm.csv.gz')

## Glorot Uniform

*Glorot normal initializer, also called Xavier normal initializer.*

## Dropout

***“to prevent over-fitting”***

![](appendix/dropout.png)

*Source: Srivastava, Nitish, et al. ”Dropout: a simple way to prevent neural networks from
overfitting”, JMLR 2014*

## RMSProp

*Gradient descent is one of the most popular algorithms to perform optimization and by far the most common way to optimize neural networks.*

![](appendix/rmsprop.gif)

*Source: http://ruder.io/content/images/2016/09/saddle_point_evaluation_optimizers.gif*

## LSTM Encoder & Decoder Parts

In [3]:
num_encoder_tokens = lstm_train['X_lstm'].shape[2]
latent_dim = 256

# encoder training
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, 
               batch_input_shape = (1, None, num_encoder_tokens),
               stateful = False,
               return_sequences = True,
               return_state = True,
               recurrent_initializer = 'glorot_uniform')

encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c] # 'encoder_outputs' are ignored and only states are kept.

# Decoder training, using 'encoder_states' as initial state.
decoder_inputs = Input(shape=(None, num_encoder_tokens))

decoder_lstm_1 = LSTM(latent_dim,
                      batch_input_shape = (1, None, num_encoder_tokens),
                      stateful = False,
                      return_sequences = True,
                      return_state = False,
                      dropout = 0.2,
                      recurrent_dropout = 0.2) # True

decoder_lstm_2 = LSTM(128,
                     stateful = False,
                     return_sequences = True,
                     return_state = True,
                     dropout = 0.2,
                     recurrent_dropout = 0.2)

decoder_outputs, _, _ = decoder_lstm_2(decoder_lstm_1(decoder_inputs, initial_state = encoder_states))
decoder_dense = TimeDistributed(Dense(lstm_train['Y_lstm'].shape[2], activation = 'relu'))
decoder_outputs = decoder_dense(decoder_outputs)

# training model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
training_model.compile(optimizer = 'rmsprop', loss = 'mse')

In [4]:
# GENERATOR APPLIED TO FEED ENCODER AND DECODER ---------------------------
# generator that randomly creates times series of 39 consecutive days
# theses time series has following 3d shape: 829 restaurants * 39 days * num_features 
def dec_enc_n_days_gen(X_3d, Y_3d, length):
    while 1:
        decoder_boundary = X_3d.shape[1] - length - 1
        
        encoder_start = np.random.randint(0, decoder_boundary)
        encoder_end = encoder_start + length
        
        decoder_start = encoder_start + 1
        decoder_end = encoder_end + 1
        
        X_to_conc = X_3d[:, encoder_start:encoder_end, :]
        Y_to_conc = Y_3d[:, encoder_start:encoder_end, :]
        X_to_decode = X_3d[:, decoder_start:decoder_end, :]
        Y_decoder = Y_3d[:, decoder_start:decoder_end, :]
        
        yield([X_to_conc,
               X_to_decode],
               Y_decoder)

training_model.fit_generator(dec_enc_n_days_gen(lstm_train['X_lstm'][:,:,:], lstm_train['Y_lstm'][:,:,:], 39),
                            steps_per_epoch = lstm_train['X_lstm'][:,:,:].shape[0],
                            verbose = 1,
                            epochs = 6)

Epoch 1/6
829/829 [==============================] - 168s 202ms/step - loss: 0.2623
Epoch 2/6
829/829 [==============================] - 166s 200ms/step - loss: 0.1688
Epoch 3/6
829/829 [==============================] - 166s 200ms/step - loss: 0.1418
Epoch 4/6
829/829 [==============================] - 166s 200ms/step - loss: 0.1259
Epoch 5/6
829/829 [==============================] - 166s 200ms/step - loss: 0.1129
Epoch 6/6
829/829 [==============================] - 166s 200ms/step - loss: 0.1080


## Predict Mechanism



In [5]:
# PREDICTION FUNCTION --------------------------------------------------

# function takes 39 days before first prediction day (input_seq)
# then using encoder to identify hidden states for these 39 days.
# Next, decoder takes hidden states provided by encoder
# and predicts number of visitors from day 2 to day 40.
# Day 40 is the first day of target_seq.

# Predicted value for day 40 is appended to features of day 41.
# Then function takes period from day 2 to day 40 and repeat the process
# unil all days in target sequence get their predictions. 

# The output of the function is the vector with predictions that has
# following shape: 820 restaurants * 39 days * 1 predicted visitors amount

def predict_sequence(inf_enc, inf_dec, input_seq, Y_input_seq, target_seq):
    # state of input sequence produced by encoder
    state = inf_enc.predict(input_seq)
    
    # restrict target sequence to the same shape as X_lstm_test
    target_seq = target_seq[:,:, :lstm_test['X_lstm_test'].shape[2]]
    
    
    # create vector that contains y for previous 7 days
    t_minus_seq = np.concatenate((Y_input_seq[:,-1:,:], input_seq[:,-1:, lstm_test['X_lstm_test'].shape[2]:-1]), axis = 2)
    
    # current sequence that is going to be modified each iteration of the prediction loop
    current_seq = input_seq.copy()
    
    
    # predicting outputs
    output = np.ones([target_seq.shape[0],1,1])
    for i in range(target_seq.shape[1]):
        # add visitors for previous 7 days into features of a new day
        new_day_features = np.concatenate((target_seq[:,i:i+1,:], t_minus_seq[...]), axis = 2)
        
        # move prediction window one day forward
        current_seq = np.concatenate((current_seq[:,1:,:], new_day_features[:,]), axis = 1)
        
        
        # predict visitors amount
        pred = inf_dec.predict([current_seq] + state)
        
        # update t_minus_seq
        t_minus_seq = np.concatenate((pred[:,-1:,:], t_minus_seq[...]), axis = 2)
        t_minus_seq = t_minus_seq[:,:,:-1]        
        
        # update predicitons list
        output = np.concatenate((output[...], pred[:,-1:,:]), axis = 1)
        
        # update state
        state = inf_enc.predict(current_seq)
    
    return output[:,1:,:]

In [6]:
# INFERENCE ENCODER AND DECODER -----------------------------------------    
# inference encoder
encoder_model = Model(encoder_inputs, encoder_states)

# inference decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs,_,_ = decoder_lstm_2(decoder_lstm_1(decoder_inputs,
                                                    initial_state = decoder_states_inputs))
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs])

## Preparing Submission & Submiting with Kaggle API

In [7]:
# Predicting test values
enc_dec_pred = predict_sequence(encoder_model,
                                decoder_model,
                                lstm_train['X_lstm'][:,-lstm_test['X_lstm_test'].shape[1]:,:],
                                lstm_train['Y_lstm'][:,-lstm_test['X_lstm_test'].shape[1]:,:],
                                lstm_test['X_lstm_test'][:,:,:])

# Add predicted test values to submission dataset ---------------------

# Note: it is important to preserve the order of time series.
# Thus, test set will contain all 829 lines in the same order as train set.
# To make this 'air_store_id' is taken as in X and not in X_test (second line of 'test' variable below).
# Only relevant results will be merged for submission dataframe
test = df_to_reshape.loc[df_to_reshape['visit_date'].isin(X_test['visit_date'].values) &
                         df_to_reshape['air_store_id'].isin(X['air_store_id'].values),]


# reshape predicted values to initial shape
test_pred = enc_dec_pred.reshape(test.shape[0], 1)
test_pred_exp = np.exp(test_pred) - 1.0
test_pred_exp[test_pred_exp<0] = 0

# add predictions to dataframe with 'air_store_id' and 'visit_date'
test_df_pred = test[['air_store_id', 'visit_date']].copy()
test_df_pred['predicted'] = test_pred_exp

# reverse transform of 'air_store_id'
test_df_pred['air_store_id'] = lstm_test['air_store_id']

# finalizing submission csv file
submission_df = submission_lstm.merge(test_df_pred,
                                     how = 'left',
                                     left_on = ['air_store_id', 'visit_date'],
                                     right_on = ['air_store_id', 'visit_date'])

submission_df['visitors'] = submission_df['predicted']
submission_df = submission_df.drop(['air_store_id', 'visit_date', 'predicted'], axis = 1)
submission_df.to_csv('submissions/lstm_submission.csv.gz', compression='gzip', index=False)

In [1]:
# !kaggle competitions submit -c recruit-restaurant-visitor-forecasting -f submissions/lstm_submission.csv.gz -m "Final LSTM" 

## References

### Encoder-Decoder Model
- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
- https://blog.keras.io/building-autoencoders-in-keras.html
- http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture13.pdf
- https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/
- https://github.com/Arturus/kaggle-web-traffic

- https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5
- http://ruder.io/optimizing-gradient-descent/